In [14]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import torch
from torch_geometric.utils import from_networkx
from torch_geometric.data import Data
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
import pandas as pd
import ast

# Attempting to create a GNN for the data

## Step 1: Load data

In [54]:
# load csvs
w1 = pd.read_csv('week1.csv')
w2 = pd.read_csv('week2.csv')
w3 = pd.read_csv('week3.csv')

In [22]:
w1.head()

,Unnamed: 0,source,genres,img_info,popularity,followers,collaborators,in_playlist,link,name,api_link,pagerank,closeness_cent,clustering,deg_cent,degree
0,Hanumankind,playlist,"['desi hip hop', 'indian underground rap', 'ma...",[{'url': 'https://i.scdn.co/image/ab6761610000...,76,503600,"{'Hanumankind': {'count': 14, 'uri': 'spotify:...",True,https://api.spotify.com/v1/artists/4nVa6XlBFlI...,Hanumankind,NaN,0.000159,0.258736,0.081871,0.000525,19
1,Kalmi,playlist,[],[{'url': 'https://i.scdn.co/image/ab6761610000...,75,32646,"{'Hanumankind': {'count': 3, 'uri': 'spotify:a...",True,https://api.spotify.com/v1/artists/6tYCSHLtTI5...,Kalmi,NaN,0.000145,0.258189,0.080882,0.000470,17
2,Parimal Shais,playlist,['malayalam hip hop'],[{'url': 'https://i.scdn.co/image/ab6761610000...,51,38998,"{'Parimal Shais': {'count': 17, 'uri': 'spotif...",False,NaN,Parimal Shais,NaN,0.000130,0.257790,0.114286,0.000415,15
3,Shalmali Kholgade,playlist,"['filmi', 'modern bollywood']",[{'url': 'https://i.scdn.co/image/ab6761610000...,63,155041,"{'Shalmali Kholgade': {'count': 20, 'uri': 'sp...",False,NaN,Shalmali Kholgade,NaN,0.000199,0.207056,0.008658,0.000608,22
4,Vishal Dadlani,playlist,"['filmi', 'modern bollywood']",[{'url': 'https://i.scdn.co/image/ab6761610000...,75,11840425,"{'Vishal Dadlani': {'count': 20, 'uri': 'spoti...",False,NaN,Vishal Dadlani,NaN,0.000265,0.206633,0.000000,0.000802,29


In [43]:
def get_weight(row:pd.Series):
    node_2 =row['edges']
    lookup = row['collaborators']
    if lookup=={}:
        return None
    return int(lookup[node_2]['count'])



In [51]:
def load_graph_fromcsv(df:pd.DataFrame):
    df.collaborators.fillna('{}',inplace=True)
    df['collaborators']=df.collaborators.apply(lambda x: ast.literal_eval(x))
    df['edges']=df.collaborators.apply(lambda x: list(x.keys()))
    df=df.explode('edges')
    df['weight']=df.apply(lambda x: get_weight(x),axis=1)
    return nx.from_pandas_edgelist(df=df,source='name',target='edges',edge_attr='weight')

In [55]:
w1_g = load_graph_fromcsv(w1)
w2_g = load_graph_fromcsv(w2)
w3_g = load_graph_fromcsv(w3)

In [58]:
# get features
w1_f = w1[['name','genres','popularity','followers']]
w2_f = w2[['name','genres','popularity','followers']]
w3_f = w3[['name','genres','popularity','followers']]
# get labels
w1_f = w1[['name','in_playlist']]
w2_f = w2[['name','in_playlist']]
w3_f = w3[['name','in_playlist']]

In [56]:
w1.head()

,Unnamed: 0,source,genres,img_info,popularity,followers,collaborators,in_playlist,link,name,api_link,pagerank,closeness_cent,clustering,deg_cent,degree,edges
0,Hanumankind,playlist,"['desi hip hop', 'indian underground rap', 'ma...",[{'url': 'https://i.scdn.co/image/ab6761610000...,76,503600,"{'Hanumankind': {'count': 14, 'uri': 'spotify:...",True,https://api.spotify.com/v1/artists/4nVa6XlBFlI...,Hanumankind,NaN,0.000159,0.258736,0.081871,0.000525,19,"[Hanumankind, Kalmi, Parimal Shais, Shalmali K..."
1,Kalmi,playlist,[],[{'url': 'https://i.scdn.co/image/ab6761610000...,75,32646,"{'Hanumankind': {'count': 3, 'uri': 'spotify:a...",True,https://api.spotify.com/v1/artists/6tYCSHLtTI5...,Kalmi,NaN,0.000145,0.258189,0.080882,0.000470,17,"[Hanumankind, Kalmi, TATA.ev, Murane Athma, KA..."
2,Parimal Shais,playlist,['malayalam hip hop'],[{'url': 'https://i.scdn.co/image/ab6761610000...,51,38998,"{'Parimal Shais': {'count': 17, 'uri': 'spotif...",False,NaN,Parimal Shais,NaN,0.000130,0.257790,0.114286,0.000415,15,"[Parimal Shais, Reble, Fejo, Dhp, DG IMMORTALS..."
3,Shalmali Kholgade,playlist,"['filmi', 'modern bollywood']",[{'url': 'https://i.scdn.co/image/ab6761610000...,63,155041,"{'Shalmali Kholgade': {'count': 20, 'uri': 'sp...",False,NaN,Shalmali Kholgade,NaN,0.000199,0.207056,0.008658,0.000608,22,"[Shalmali Kholgade, Anusha Mani, Harshdeep Kau..."
4,Vishal Dadlani,playlist,"['filmi', 'modern bollywood']",[{'url': 'https://i.scdn.co/image/ab6761610000...,75,11840425,"{'Vishal Dadlani': {'count': 20, 'uri': 'spoti...",False,NaN,Vishal Dadlani,NaN,0.000265,0.206633,0.000000,0.000802,29,"[Vishal Dadlani, Salim Merchant, Shekhar Ravji..."
